
## KCI 사회과학 분야 한국어 초록 분석(2004–2024) 


# 1) 데이터 로드 및 전처리

In [ ]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv(r'/home/work/KCI_LLM_Lab/raw data/사회과학_korean_papers_2004-2024_20251002T195849.csv')  # 2004–2024년 전체 데이터

# 불러온 데이터 정보 확인
print("데이터 크기:", df.shape)
print("\n컬럼 목록:", list(df.columns))
print("\n데이터 미리보기 (앞 3줄):")
print(df.head(3))

In [ ]:
# 사용하고자 하는 컬럼 추출
df = df[['논문ID', '발행년도', '논문제목', '초록', '주제분야']]
print(f"필터링된 데이터프레임 크기: {df.shape}")

# 결측치 확인
print("\n결측치 확인:")
print(df.isnull().sum())

In [ ]:
# 공백/빈문자열 및 결측 문자열 → NA 처리
for c in ['논문제목', '초록']:
    s = df[c].astype(str).str.strip().str.lower()
    # "nan", "none", "null", "<na>", "n/a" 같은 문자열 결측 토큰 제거
    s = s.replace(r'^(nan|none|null|<na>|n/a)$', '', regex=True)
    # 빈 문자열을 NA로 바꾸기
    s = s.replace({'': pd.NA})
    df[c] = s

# 초록만 필수로 남기기
df = df.dropna(subset=['초록'])
print(f"[OK] 초록 결측 제거 후 크기: {df.shape}")

# 데이터 정보 확인
print(df.shape)
print(df.columns)
df.head()

### 추후 주제별 비교를 위해 '주제분야' 컬럼 결측 보정 및 계층 분해

In [ ]:
import pandas as pd
import re

# 결측/유사결측 정의
NULL_LIKE = {'', 'nan', 'none', 'null', 'na', '미기재', 'N/A'}

# 문자열 표준화: 공백 정리, 구분자 ' > '로 통일
s = df['주제분야'].astype(str).str.strip()

# '인문학  >  역사학' 같은 경우 공백 정리 및 다양한 기호 통일
s = s.replace(r'\s*>\s*', ' > ', regex=True)   # 구분자 좌우 공백 통일
s = s.replace(r'\s+', ' ', regex=True)         # 다중 공백 → 단일 공백

# 유사결측을 NA로 치환
s_lower = s.str.lower()
s = s.where(~s_lower.isin({x.lower() for x in NULL_LIKE}), pd.NA)

# 결측 채움(분석 편의용 라벨)
s = s.fillna('미상')

# 계층 분해: 대/중/소 분류 (최대 3단계로 가정함, 더 깊으면 마지막에 합침)
# 단계가 많은 예시 : 인문학 > 한국어와문학 > 국어학 > 국어사
parts = s.str.split(' > ', n=2, expand=True)  # n=2 ⇒ 최대 3조각
parts.columns = ['대분류','중분류','소분류']

# 소분류에 여전히 ' > '가 남았다면(4단계 이상) 마지막 조각에 이어붙여진 상태이므로 그대로 둡니다.

# 정리된 컬럼 반영
df['주제분야_표준'] = s
df[['대분류','중분류','소분류']] = parts

# 타입: 범주형으로 변환(메모리 절약 + 그룹화 속도)
for c in ['주제분야_표준','대분류','중분류','소분류']:
    df[c] = df[c].astype('category')


In [ ]:
# 결과 저장 경로 설정
from pathlib import Path
save_dir = Path('results_S_ver4')
save_dir.mkdir(parents=True, exist_ok=True)

# 1. 연도별 데이터 개수
year_counts = df['발행년도'].value_counts().sort_index()
print("연도별 논문 수:")
print(year_counts)
year_counts.to_csv(save_dir / 'SS_yearly_counts.csv', encoding='utf-8-sig')

# 2. 연도별 중분류 개수
year_category_counts = df.groupby('발행년도')['중분류'].nunique()
print("\n연도별 중분류 개수:")
print(year_category_counts)
year_category_counts.to_csv(save_dir / 'SS_yearly_category_counts.csv', encoding='utf-8-sig')

# 3. 중분류별 논문 수 집계
category_stats = df['중분류'].value_counts()
print("\n중분류별 논문 수:")
print(category_stats)
category_stats.to_csv(save_dir / 'SS_category_paper_counts.csv', encoding='utf-8-sig')

# 저장 완료 메시지
print("\n✅ 저장 완료:")
print(f" - 연도별 논문 수: {save_dir}/SS_yearly_counts.csv")
print(f" - 연도별 중분류 수: {save_dir}/SS_yearly_category_counts.csv")
print(f" - 중분류별 논문 수: {save_dir}/SS_category_paper_counts.csv")

### 데이터 저장


In [ ]:
from pathlib import Path

# 0) 결과 폴더 생성: SS/results_S_ver2
out_dir = Path.cwd() / "results_S_ver4"
out_dir.mkdir(parents=True, exist_ok=True)

# 1) 저장 대상 DF (df 사용)
df_to_save = df.reset_index(drop=True)

# 2) 파일 경로 (파일명 앞에 SS 사회과학 접두사 붙임)
csv_path = out_dir / "SS_kor_abstract_clean.csv"
pkl_path = out_dir / "SS_kor_abstract_clean.pkl"

# 3) 저장
df_to_save.to_csv(csv_path, index=False, encoding="utf-8-sig")
df_to_save.to_pickle(pkl_path)

# 4) 로그 & 간단 점검
print("[저장 완료]")
print("Rows:", len(df_to_save))
print("CSV   →", csv_path.resolve())
print("Pickle →", pkl_path.resolve())
if "발행년도" in df_to_save.columns:
    print("Year range:", int(df_to_save["발행년도"].min()), "→", int(df_to_save["발행년도"].max()))
if "초록" in df_to_save.columns:
    print("초록 NaN 개수:", int(df_to_save["초록"].isna().sum()))

### csv 미리 보기

In [ ]:
from pathlib import Path
import pandas as pd

csv_path = Path.cwd() / "results_S_ver4" / "SS_kor_abstract_clean.csv"

# 1) 존재 확인
if not csv_path.exists():
    raise FileNotFoundError(f"파일이 없습니다: {csv_path}")

# 2) 로드 (저장 시 UTF-8-SIG)
df_loaded = pd.read_csv(csv_path, encoding="utf-8-sig", low_memory=False)

# 3) 간단 미리보기
print("[로드 완료]")
print("shape:", df_loaded.shape)
print("columns:", list(df_loaded.columns))
display(df_loaded.head(5))


In [ ]:
!pip install kiwipiepy

## 2) 형태소 분석 및 패턴 복원

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
from kiwipiepy import Kiwi
from collections import defaultdict
from tqdm import tqdm
import warnings
import multiprocessing as mp
from functools import partial
import numpy as np
from pathlib import Path
import re
warnings.filterwarnings('ignore')

# 패턴 정의
COMBINED_PATTERNS = [
    # 1. '-하게' 부사형 복원 (정확하게, 복잡하게, 세련되게)
    ("XR", "XSV", "EC"),
    ("XR", "XSA", "EC"),
    ("NNG", "XSA", "EC"),
    ("VA", "EC"),

    # 2. '-한' 관형형 복원 (화려한, 다양한, 적절한)
    ("XR", "XSA", "ETM"),
    ("NNG", "XSA", "ETM"),
    ("VA", "ETM"),

    # 3. 복합 형용사 관형형 (전형적인, 체계적인, 논리적인)
    ("NNG", "XSN", "VCP", "ETM"), # 전형 + 적 + 이 + 은

    # 4. '-적으로' 복원 (복합적으로, 이론적으로)
    ("NNG", "XSN", "JKB"),

    # 5. 핵심 복합 용언 '하다/되다' 동사/형용사 활용형 복원
    ("NNG", "XSV", "EF"),        # 종결형: 주장하다
    ("NNG", "XSV", "ETM"),       # 관형형: 주장하는
    ("NNG", "XSV", "EC"),        # 연결형: 주장하고
    ("NNG", "XSV", "EP", "EF"),  # 과거 종결형: 주장했다
    ("NNG", "XSV", "EP", "ETM"), # 과거 관형형: 주장했던
    ("NNG", "XSV", "EP", "EC"),  # 과거 연결형: 주장하였고
    
# 6. 일반 용언 활용형 복원
    ("VV", "ETM"),       # 동사 어간 + 관형형 어미
    ("VA", "ETM"),       # 형용사 어간 + 관형형 어미
    ("VV", "EP", "ETM"), # 동사 과거형 + 관형형
    ("VA", "EP", "ETM"), # 형용사 과거형 + 관형형
    ("VV", "EF"),         # [추가] 동사 종결형 (사회과학 17위 / 인문학 15위)
    ("VV", "EP", "EF"),   # [추가] 동사 과거 종결형 (사회과학 27위 / 인문학 28위)
    ("VV", "EC"),         # [추가] 동사 연결형 (인문학 12위)

    # 7. 보조 용언 활용형 복원
    ("VX", "ETM"),        # 보조 용언 + 관형형 어미
    ("VX", "EP", "ETM"),  # 보조 용언 과거 관형형
    ("VX", "EF"),         # [추가] 보조용언 종결형 (인문학 24위)
]

# 결과 저장 경로 설정
save_dir = Path('/home/work/KCI_LLM_Lab/Seul/results_S_ver4')
save_dir.mkdir(parents=True, exist_ok=True)

# 산출물 경로 정의
in_pkl = save_dir / "SS_kor_abstract_clean.pkl"                   # 전처리된 입력 데이터
out_word_year_counts = save_dir / "SS_word_year_matrix_counts.csv" # 단어-연도 행렬
out_doc_counts = save_dir / "SS_doc_counts.csv"                    # 연도별 문서 수
tokenized_file = save_dir / '사회과학_tokenized_results.csv'        # 토큰화 결과

# 전처리된 데이터 불러오기
if not in_pkl.exists():
    raise FileNotFoundError(f"전처리된 파일이 없습니다: {in_pkl}")
    
df = pd.read_pickle(in_pkl)
print(f"[OK] 데이터 로드 완료: {len(df)}개 문서")

def clean_text(text):
    """텍스트 전처리: 특수문자, 문장부호 등 제거"""
    if pd.isna(text):
        return ""
        
    # 괄호와 그 내용 제거
    text = re.sub(r'\([^)]*\)', ' ', text)
    
    # 특수문자 및 문장부호 제거
    text = re.sub(r'[,.!?:;\'"\/\{\}\[\]<>~`@#$%^&*=+|\\『』「」]', ' ', text)
    
    # 여러 개의 공백을 하나로 치환
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def initialize_kiwi():
    """각 프로세스에서 Kiwi 초기화 (kiwi v0.22.0 이상에서는 cong 모델 바로 사용 가능)"""
    return Kiwi(model_type='cong')

def analyze_words(text, kiwi=None):
    """텍스트의 각 어절에 대한 형태소 분석 수행"""
    if pd.isna(text):
        return []
    
    # 텍스트 전처리
    text = clean_text(text)
    if not text:
        return []
    
    if kiwi is None:
        kiwi = initialize_kiwi()
    
    results = []
    words = text.split()
    
    for word in words:
        try:
            # 각 어절에 대한 형태소 분석
            morphs = kiwi.tokenize(word)
            morph_sequence = [(m.form, m.tag) for m in morphs]
            
            # 해당 어절의 태그 시퀀스
            tags = tuple(tag for form, tag in morph_sequence)  
            
            # 패턴 매칭 여부 확인
            matches_pattern = any(
                len(pattern) == len(tags) and all(p == t for p, t in zip(pattern, tags))
                for pattern in COMBINED_PATTERNS
            )
            
            # 모든 어절에 대해 형태소 분석 결과 저장
            results.append({
                'word': word,                  # 원본 어절
                'morphs': morph_sequence,      # 형태소 분석 결과
                'matches_pattern': matches_pattern  # 패턴 매칭 여부
            })
            
        except Exception as e:
            print(f"형태소 분석 중 오류 발생: {str(e)}")
            continue
            
    return results

def process_chunk(texts):
    """청크 단위로 토큰화 처리"""
    kiwi = initialize_kiwi()
    return [analyze_words(text, kiwi) for text in texts]

def parallel_tokenize(texts, n_cores):
    """병렬 처리로 토큰화 수행"""
    chunk_size = len(texts) // n_cores + 1
    chunks = [texts[i:i + chunk_size] for i in range(0, len(texts), chunk_size)]
    
    with mp.Pool(n_cores) as pool:
        results = list(tqdm(
            pool.imap(process_chunk, chunks),
            total=len(chunks),
            desc="청크 처리 중"
        ))
    
    # 결과 합치기
    return [token for chunk_result in results for token in chunk_result]

# CPU 코어 수 확인 (전체 코어의 75% 사용)
num_cores = max(1, int(mp.cpu_count() * 0.75))
print(f"\n사용할 CPU 코어 수: {num_cores}")

# Kiwi 초기화 확인
print("\nKiwi 초기화 중...")
test_kiwi = Kiwi(model_type='cong')
print("CoNG 모델 로드 완료")

# 병렬 토큰화 수행
print("\n초록 토큰화 시작...")
df['tokens'] = parallel_tokenize(df['초록'].tolist(), num_cores)

# 토큰화 결과 저장
print("\n토큰화 결과 저장 중...")
df.to_csv(tokenized_file, index=False, encoding='utf-8-sig')

# 연도별 단어 출현 빈도 계산 (COMBINED_PATTERNS에 매칭되는 어절만)
print("\n연도별 단어 빈도 계산 중...")
word_year_counts = defaultdict(lambda: defaultdict(int))
word_is_pattern = {}  # 단어가 패턴에 맞는지 여부 저장
years = sorted(df['발행년도'].unique())

for _, row in tqdm(df.iterrows(), total=len(df), desc="단어 빈도 계산"):
    year = row['발행년도']
    tokens = row['tokens']
    
    for token in tokens:
        if token['matches_pattern']:
            # 패턴에 맞는 경우만 어절 단위로 저장
            word = token['word']
            word_year_counts[word][year] += 1
            word_is_pattern[word] = True
        # 패턴에 맞지 않는 경우는 제외 (결과에 포함하지 않음)

# 데이터프레임으로 변환
word_year_df = pd.DataFrame.from_dict(word_year_counts, orient='index')
word_year_df = word_year_df.reindex(columns=years, fill_value=0)

# 패턴 매칭 여부 열 추가
word_year_df['matches_pattern'] = pd.Series(word_is_pattern)
word_year_df.to_csv(out_word_year_counts, encoding='utf-8-sig')

# 연도별 전체 문서 수 저장
doc_counts = df['발행년도'].value_counts().sort_index()
doc_counts.to_frame(name='n_docs').to_csv(out_doc_counts, encoding='utf-8-sig')

print("\n✅ 완료:")
print(f" - 토큰화 결과: {tokenized_file}")
print(f" - 단어-연도 행렬: {out_word_year_counts}")
print(f" - 연도별 문서 수: {out_doc_counts}")
print(f" - 고유 단어 수: {len(word_year_counts)}")
print(f" - 전체 문서 수: {len(df)}")

# 빈도수 상위 20개 단어와 그들의 패턴 매칭 여부 출력
total_counts = word_year_df.drop('matches_pattern', axis=1).sum(axis=1)
top_20 = total_counts.nlargest(20)
print("\n가장 빈번한 단어 20개:")
for word, count in top_20.items():
    is_pattern = word_year_df.loc[word, 'matches_pattern']
    pattern_str = "패턴 매칭" if is_pattern else "개별 형태소"
    print(f"{word}: {count}회 ({pattern_str})")

# 3) 연도별 빈도 계산 및 2024년 과잉 어휘 식별

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# ------------------------------
# 0) 경로
# ------------------------------

base_dir = Path("results_S_ver4")
counts_path = base_dir / "SS_word_year_matrix_counts.csv"
docs_path   = base_dir / "SS_doc_counts.csv"

out_full    = base_dir / "SS_excess_words_2024.csv"          # 2024년 분석 대상 모든 단어의 결과
out_filtered= base_dir / "SS_excess_words_2024_filtered.csv" # 임계값 필터하여 과잉어휘만 모은 결과


# ------------------------------
# 1) 데이터 로드
# ------------------------------
counts = pd.read_csv(counts_path, index_col=0)   # 행 = terms, 열 = years
doc_counts = pd.read_csv(docs_path, index_col=0)["n_docs"]  # 인덱스 = years, 밸류 = n_docs

# 분석에 쓸 연도들
YEARS = [2021, 2022, 2023, 2024]
missing = [y for y in YEARS if str(y) not in counts.columns or y not in doc_counts.index]
if missing:
    raise ValueError(f"필요 연도 누락: {missing}")

# ------------------------------
# 2) 단어의 연도별 등장비율 p 계산,  p = (a+1)/(b+1)
# ------------------------------
p_s = {}
for y in YEARS:
    a = counts[str(y)].astype(float)
    b = float(doc_counts.loc[y])
    p_s[y] = (a + 1.0) / (b + 1.0) #a는 그 연도에 해당 단어가 등장한 문서 수, b는 그 연도 전체 문서 수
    # 즉, 그 연도 전체 문서 중 몇 %에서 해당 단어가 쓰였는가. +1은 0으로 나누는 걸 방지함

p_df = pd.DataFrame(p_s)  # p_df는 index = 단어, columns = [2021, 2022, 2023, 2024] 형태의 “연도별 등장비율 행렬”
p_df.index.name = "term" # 각 단어(term)가 2021–2024년 각 연도 초록 중 몇 %에서 등장했는가를 담은 비율 행렬

# ------------------------------
# 3) 분석 대상 단어 선택 (빈도 ≥ 1e-4 in 2024 AND 2023)
# 빈도 ≥ 1e-4는 희귀한 잡음을 제거하려는 목적이므로 단어선정의 안정성을 위해 2023년도도 포함함
# (아래의 예상치 계산에서는 2023년도는 사용하지 않는다는 점이 다름)
# ------------------------------
min_p = 1e-4
mask_keep = (p_df[2024] >= min_p) & (p_df[2023] >= min_p)
p_df = p_df.loc[mask_keep].copy()

# ------------------------------
# 4) q(2024 예상치) 계산 - 외삽 규칙
#     q = p2022 + 2*max(p2022 - p2021, 0)   # (증가 추세면 2배 증가, 감소 추세면 현상 유지)
# -> 하락한 단어는 증가하지 않는다는 보수적 가정, 즉, q >= p2022
# -> 상승한 단어는 증가 추세를 반영해서 2배 반영, q를 높게 잡아 δ = p − q가 부풀려지지 않도록(보수적 하한) 설계
# ------------------------------
p21 = p_df[2021].values
p22 = p_df[2022].values
p24 = p_df[2024].values

trend = np.maximum(p22 - p21, 0.0)
q = p22 + 2.0 * trend
q = np.maximum(q, p22)  # 보수적 하한

# ------------------------------
# 5) δ, ratio, log10 r 계산
# ------------------------------
eps = 1e-12
delta = p24 - q
ratio = p24 / np.maximum(q, eps)
log10r = np.log10(np.maximum(ratio, eps))

# ------------------------------
# 6) 결과 테이블 구성
# ------------------------------
out = pd.DataFrame({
    "p2021": p_df[2021],
    "p2022": p_df[2022],
    "p2023": p_df[2023],
    "p2024": p_df[2024],
    "q2024": q,
    "delta": delta,
    "ratio": ratio,
    "log10_ratio": log10r,
})

# 정렬: δ와 ratio 모두 큰 순으로 참고하기 좋도록
out_sorted = out.sort_values(["delta", "ratio"], ascending=[False, False])

# 전체 저장
out_sorted.to_csv(out_full, encoding="utf-8-sig")

# ------------------------------
# 7) 임계값 필터 (수정 버전)
#    - δ > 0.01  또는  log10 r > 0.1 (고정 컷)
#    - + 빈도 필터: p2024, p2023 ≥ 1e-4
# ------------------------------
gap_thr = 0.01          # 절대적 빈도 차이 기준; 0.01 = 1%p 이상의 과잉만 채택
logr_thr = 0.1          #  log10(r) 고정 컷; 너무 많은 단어가 통과하지 않도록 보수적으로 설정
min_freq = 1e-4         # 분석 연도(Y)와 직전 연도(Y-1) 모두 p ≥ 1e-4인 단어만 대상
eps = 1e-12             # 수치 안정성용 아주 작은 값(0 나눗셈, log(0) 방지)

# 빈도 필터 선적용 (Y와 Y-1에서 p ≥ 1e-4)
# 극저빈도(희귀) 단어는 작은 변동에도 δ, r이 크게 출렁이며 log 기준을 쉽게 통과하므로
# 먼저 제거해 과검출을 줄임
mask_freq = (out_sorted["p2024"] >= min_freq) & (out_sorted["p2023"] >= min_freq)
cand = out_sorted[mask_freq]

# --- 임계식 적용 ---
# 원논문과 다른 점은 log 기준을 고정값(0.1)로 보수화하여 과검출을 억제함.
#  원 논문의 log 컷은 대략 0.075(≈19% 증가) 이며 이 분석의 경우 0.1(≈26% 증가)로 상향 조정해서 과검출(거짓양성)을 줄임.
filtered = cand[
    (cand["delta"] > gap_thr) |
    (cand["log10_ratio"] > logr_thr)
]

# 임계값 통과 단어 수 (필터 전/후 모두 표기)
print(f"임계값 통과 단어 수: {len(filtered)} / {len(cand)} (원본 {len(out_sorted)})")

# 저장
filtered.to_csv(out_filtered, encoding="utf-8-sig")

print("✅ 완료:")
print(" - 전체 표  :", out_full)
print(" - 임계값 표:", out_filtered)  # 최종 Excess words 리스트
print(" - 대상 단어 수(원본):", len(out_sorted))
print(" - 빈도 필터 후 대상:", len(cand))
print(" - 임계 통과 수     :", len(filtered))



# 4)  과잉어휘 목록 생성

2025.12.25 Ratio 나열시 최소 문서수 30이상 기준으로 필터링

In [ ]:
# delta 및 ratio 기준 상위 과잉어휘 100 + 100 병합 --> 파일 생성
import pandas as pd
from pathlib import Path

base = Path("results_S_ver4")
filtered = pd.read_csv(base / "SS_excess_words_2024_filtered.csv")

# 인덱스 복원/표준화: 'term' → 'word' 로 통일
filtered = filtered.reset_index(drop=True)
if "word" not in filtered.columns:
    if "term" in filtered.columns:
        filtered = filtered.rename(columns={"term": "word"})
    elif "index" in filtered.columns:
        filtered = filtered.rename(columns={"index": "word"})
    elif "Unnamed: 0" in filtered.columns:
        filtered = filtered.rename(columns={"Unnamed: 0": "word"})
    else:
        filtered = filtered.reset_index().rename(columns={"index": "word"})

# 숫자 컬럼 보정 (안전)
for col in ("delta", "ratio", "p2024"):
    if col in filtered.columns:
        filtered[col] = pd.to_numeric(filtered[col], errors="coerce")

# -------------------------
# 2024 실제 문서 수 기반 df 계산
# -------------------------
N_DOCS_2024 = 25198
filtered["df2024"] = (filtered["p2024"] * N_DOCS_2024).round().astype("Int64")

# -------------------------
# ratio 필터 기준: 최소 문서수
# -------------------------
MIN_DOCS = 30  # 수정 해 가면서 볼 수 있음(df=10·20·30을 비교한 민감도 점검 결과, 
# df=30은 희귀 단어들이 ratio 상위에 포함되는 현상을 억제하면서도, 
# 반복적으로 사용된 표현을 보존하는 최소 기준으로 적절해 보임)
ratio_pool = filtered[filtered["df2024"] >= MIN_DOCS].copy()

print(f"✅ ratio 후보군: {len(ratio_pool)} / {len(filtered)} (df2024 >= {MIN_DOCS})")

# 전체 데이터 기준 delta 랭킹만 계산 (참고용)
tmp = filtered.copy()
tmp["delta_rank"] = tmp["delta"].rank(ascending=False, method="min")

# Top100 추출
top_delta = filtered.nlargest(100, "delta")
top_ratio = ratio_pool.nlargest(100, "ratio")  # <-- df 필터 통과한 후보군에서만 추출

# 합집합 + 중복 제거
combined = pd.concat([top_delta, top_ratio], ignore_index=True).drop_duplicates("word")

# delta_rank만 병합 (전체 데이터 기준 순위 유지)
combined = combined.merge(tmp[["word", "delta_rank"]], on="word", how="left")

# ratio_rank는 combined 데이터 내에서 새로 계산 (1~100 순위)
combined["ratio_rank"] = combined["ratio"].rank(ascending=False, method="min")

# 라벨 컬럼 없으면 생성
if "manual_label" not in combined.columns:
    combined["manual_label"] = ""   # 이후 수동 라벨링: style/content 등

# 출력 파일명
out_csv   = base / "SS_excess_words_top_combined_for_labeling_final.csv"
out_excel = base / "SS_excess_words_top_combined_for_labeling_final.xlsx"

combined.to_csv(out_csv, index=False, encoding="utf-8-sig")
combined.to_excel(out_excel, index=False)

print(f"✅ 총 {len(combined)}개 단어가 추출되어 저장되었습니다. → {out_csv}")
print(f"✅ 엑셀 파일로도 저장되었습니다. → {out_excel}")


이후 한글 초록 분석의 시각화는 영어 분석 코드(English_analysis.ipynb)로 입력 데이터만 변경하여 진행하였음